 **Algal Bloom Detection in Lake Erie using MODIS Aqua Imagery**

MODIS MYDOCGA V6 ocean reflectance product is used for detecting algal bloom. It consists of 1 kilometer reflectance data from Aqua MODIS bands 8-16. The bands 15, 14, 13 is used as RGB combination. Bands 15,14,13 of MODIS Aqua are 10nm windows of 743–753nm,673–683nm and 662–672nm respectively, hence the pixels in each band holds mixed values of surface reflectance across these nm windows.

In [16]:
#! pip install geemap

In [2]:
# Importing libraries

import ee
import geemap

In [13]:
# Initializing EarthEngine API
#ee.Authenticate()

# Initialize the library.
#ee.Initialize()

In [4]:
# Defining area of interest (Lake Erie)

Map = geemap.Map()

aoi = ee.Geometry.Polygon([
[-83.5029831043747,41.69766662039789],
[-83.3821334949997,41.66894932807228],
[-83.1184616199997,41.5991539797965],
[-82.9536666981247,41.500490731494615],
[-83.0470504871872,41.45933660936861],
[-83.0140915028122,41.385193340909105],
[-82.8218307606247,41.43463159259681],
[-82.7723922840622,41.426394496647724],
[-82.5032272449997,41.3687055638583],
[-82.1516647449997,41.47168559004283],
[-81.9923629871872,41.47991693718029],
[-81.7286911121872,41.463453197582886],
[-81.5968551746872,41.545730078233376],
[-81.4320602528122,41.652533692609516],
[-81.1299362293747,41.7960285924172],
[-80.8717575184372,41.86970115960587],
[-80.3334274403122,42.016791844815],
[-79.1963424793747,42.488440945761944],
[-79.0425338856247,42.67045833361161],
[-78.8283004871872,42.767316978993314],
[-78.8997116199997,42.90427453361668],
[-79.6357956043747,42.87610220414661],
[-80.2620163074997,42.767316978993314],
[-80.4817428699997,42.58153843881767],
[-80.6520309559372,42.61388764145159],
[-80.9926071278122,42.678535661975104],
[-81.3386764637497,42.66237995542755],
[-81.6408004871872,42.50869133669173],
[-82.0527877918747,42.261188632038866],
[-82.4867477528122,42.08613336096329],
[-82.5306930653122,41.93920293478214],
[-82.5911178699997,42.04535340136033],
[-82.9536666981247,41.99230029652944],
[-83.1349411121872,42.14318129356413],
[-83.1733932606247,42.11466374603782],
[-83.2777633778122,41.96371490826293],
[-83.4425582996872,41.82059553649974],
[-83.5029831043747,41.69766662039789]
])
Map.setCenter(-81.3399, 42.0669, 8);
Map

Map(center=[42.0669, -81.3399], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

The bands 15, 14, 13 are chosen as RGB combination because of apparent 'red edge' of harmful algal bloom in the infra red region of the electromagnetic spectrum. This give very nice red color for blooms but other sediment rich areas can also show up in red. So that needs to be removed. 

Bands 15,14,13 of MODIS Aqua are 10nm windows of 743–753nm,673–683nm and 662–672nm respectively, hence the pixels in each band holds mixed values of surface reflectance across these nm windows. 

Also, on Google Earth Engine the pixel values are scaled with a multiplication by 10000

In [9]:
Map = geemap.Map()


Map.setCenter(-81.3399, 42.0669, 8);


# Defininf MODIS aqua imagery collection


#Clipping MODIS imagery to area of interest 
modis_img = ee.ImageCollection('MODIS/006/MYDOCGA').filter(ee.Filter.date('2021-04-06','2021-04-07')).first().clip(aoi) 

# Bands 15, 14, and 13 represents R, G and B respectively
FalseColorVis = {
  'min': -50,
  'max': 100,
  'bands':['sur_refl_b15','sur_refl_b14','sur_refl_b13']
}

# Creating a FasleColorBlooms layer
Map.addLayer(modis_img, FalseColorVis, 'FasleColorBlooms')

Map

Map(center=[42.0669, -81.3399], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

The reflectance values of bands 15 and 13 have inverse relationship which can be used to perform simple math calculation to bring out algar bloom from an image. 

By declaring the variable, we can see that when the difference between the reflectance values of Band 15 and Band 13 is large, then the bloom value also increases, and when its low, bloom value decreases. 

The emphasis of this relationship is more on Band 15, since Band 15 is in the Infra-red region, which is an indication that higher the Infra-red reflectance, more thick and harmful the bloom. 

In [10]:
Map = geemap.Map()


Map.setCenter(-81.3399, 42.0669, 8);


# Defining variables 'x' and 'y' as bands 15 and 13 


x = modis_img.select('sur_refl_b15')
y = modis_img.select('sur_refl_b13')
z = (y).subtract(x)
algal_bloom = ((x.subtract(z)))

viz = {
  'min': 0,
  'max': 30,
  'palette': ['cyan','green','yellow','orange','red']
}

Map.addLayer(algal_bloom, viz, 'Initial Bloom Layer')

Map

Map(center=[42.0669, -81.3399], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [15]:
# Applying the mask

maskedBloom = algal_bloom.updateMask(Blooms.gt(8))
maskedBloom = maskedBloom.updateMask(maskedBloom.lt(30))
viz2 = {
  'min': 8,
  'max': 30,
  'palette': ['cyan','green','yellow','orange','red']
}
Map.addLayer(maskedBloom, viz2, 'Harmful Blooms')
Map

Map(bottom=24612.0, center=[42.0669, -81.3399], controls=(WidgetControl(options=['position', 'transparent_bg']…

In Quality Control QC band all Bits layers which are numbered as ‘0’ represents the best quality pixels, so we mask our ‘maskedBloom’ layer with all the QC layers other than 0, and simply overwrite it, so that we only view the best pixels.

In [14]:
# Removing the noise by applying Quality Control QC band

qc = modis_img.select('QC_b8_15_1km')
maskedBloom = maskedBloom.updateMask(qc.neq(0))
Map.addLayer(maskedBloom, viz2, 'QC Harmful Blooms')
Map


Map(bottom=24612.0, center=[42.0669, -81.3399], controls=(WidgetControl(options=['position', 'transparent_bg']…